In [8]:
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from skimage.io import imread
from shutil import copyfile
#import tensorflow_probability as tfp
print(tf.__version__)
import matplotlib.pyplot as plt
plt.gray()
from sklearn.metrics import accuracy_score
#import tensorflow_probability as tfp
import random
import json
import datetime


BATCH_SIZE = int(32.)
EPOCHS = int(512.)

batch_size = BATCH_SIZE
epochs = EPOCHS
learning_rate=0.001
CHANNELS=1
NLAYERS=int(1.)
NFILTERS=int(48.)
KSIZE=13
SUBSPACE=12
PATIENCE_ES=40
PATIENCE_RP=5


class NeymanScott:
    """
    Neyman-Scott point process using a Poisson variable for the number of parent points, uniform for
    the number of daughter points and Pareto distribution for the distance from the daughter points to
    the parent.
    """
    def __init__(self,
                 poisson_mean: float,
                 daughter_max: int,
                 pareto_alpha: float,
                 pareto_scale: float,
                 size: (int, int)):
        """
        :param poisson_mean: mean of the number of parent points
        :param daughter_max: maximum number of daughters per parent points
        :param pareto_alpha: alpha parameter of the Pareto distribution
        :param pareto_scale: scale used in the Pareto distribution. This parameter is
            applied before resizing the points from the [0, 1] interval to the size of the image.
        :param size: rescale the output to this size
        """
        self.poisson_mean = poisson_mean
        self.daughter_max = daughter_max
        self.pareto_alpha = pareto_alpha
        self.pareto_scale = pareto_scale
        self.size = np.array([size])
        self.generator = np.random.Generator(np.random.PCG64())

    def __call__(self):
        num_parents = self.generator.poisson(lam=self.poisson_mean)
        parents = self.generator.random((num_parents, 2))
        num_daughters = self.generator.integers(1, self.daughter_max, num_parents)
        points = np.empty((0, 2))

        for i in range(num_parents):
            # normalizes the pareto II distribution
            dist = self.generator.pareto(self.pareto_alpha, (num_daughters[i], 1))
            dist = (dist + 1) * self.pareto_scale
            angle = self.generator.uniform(0., 2 * np.pi, (num_daughters[i],))
            positions = np.stack([np.cos(angle), np.sin(angle)], 1)
            positions *= dist
            positions += parents[i, np.newaxis, :]
            points = np.concatenate([points, positions])
        # remove points outside the set [0, 1] x [0, 1]
        valid_points = np.logical_and(
            np.logical_and(0. <= points[:, 0], points[:, 0] <= 1.),
            np.logical_and(0. <= points[:, 1], points[:, 1] <= 1.)
        )
        points = points[valid_points, :]
        # scale to the image size
        points = points * self.size
        return points



NSAMPLES_TRAINING=2024*2
IMG_SIZE=128
poisson_mean=100
daughter_max=50
pareto_scale=.02
pareto_alpha=1. #GENERATION ON IT
gen = NeymanScott(poisson_mean, daughter_max, pareto_alpha, pareto_scale, (IMG_SIZE, IMG_SIZE))

generatedata=0
if generatedata==1:
    listIm=[]
    listY=[]
    for i in range(NSAMPLES_TRAINING):
        pareto_alpha=gen.generator.random(1)*10
        gen = NeymanScott(poisson_mean, daughter_max, pareto_alpha, pareto_scale, (IMG_SIZE, IMG_SIZE))
        points = gen()
        I=np.zeros([IMG_SIZE,IMG_SIZE])
        I[np.int64(np.floor(points[:, 0])), np.int64(np.floor(points[:, 1]))]=1
        listIm.append(I)
        listY.append(pareto_alpha)
    listIm=np.stack(listIm)
    listY=np.stack(listY)

    NSAMPLES_VALIDATION=512
    listImVal=[]
    listYVal=[]
    for i in range(NSAMPLES_VALIDATION):
        pareto_alpha=gen.generator.random(1)*10
        gen = NeymanScott(poisson_mean, daughter_max, pareto_alpha, pareto_scale, (IMG_SIZE, IMG_SIZE))
        points = gen()
        I=np.zeros([IMG_SIZE,IMG_SIZE])
        I[np.int64(np.floor(points[:, 0])), np.int64(np.floor(points[:, 1]))]=1
        listImVal.append(I)
        listYVal.append(pareto_alpha)

    listImVal=np.stack(listImVal)
    listYVal=np.stack(listYVal)
    np.save('listIm.npy',listIm)
    np.save('listImVal.npy',listImVal)
    np.save('listY.npy',listY)
    np.save('listYVal.npy',listYVal)
else:
    listIm=np.load('listIm.npy')
    listImVal=np.load('listImVal.npy')
    listY=np.load('listY.npy')
    listYVal=np.load('listYVal.npy')

print('listIm.shape',listIm.shape)
print('listImVal.shape',listImVal.shape)
print('listY.shape',listY.shape)
print('listYVal.shape',listYVal.shape)

listY=listY/9
listYVal=listYVal/9


2.13.0
listIm.shape (4048, 128, 128)
listImVal.shape (512, 128, 128)
listY.shape (4048, 1)
listYVal.shape (512, 1)


<Figure size 640x480 with 0 Axes>

In [ ]:
@tf.function
def dilation2d(x, st_element, strides, padding,rates=(1, 1)):
    """

    From MORPHOLAYERS

    Basic Dilation Operator
    :param st_element: Nonflat structuring element
    :strides: strides as classical convolutional layers
    :padding: padding as classical convolutional layers
    :rates: rates as classical convolutional layers
    """
    x = tf.nn.dilation2d(x, st_element, (1, ) + strides + (1, ),padding.upper(),"NHWC",(1,)+rates+(1,))
    return x

class DepthwiseDilation2D(Layer):
    '''
    Depthwise Dilation 2D Layer: Depthwise Dilation for now assuming channel last
    '''
    def __init__(self, kernel_size,depth_multiplier=1, strides=(1, 1),padding='same', dilation_rate=(1,1), kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1., maxval=0.),
    kernel_constraint=None,kernel_regularization=None,**kwargs):
        super(DepthwiseDilation2D, self).__init__(**kwargs)
        self.kernel_size = kernel_size
        self.depth_multiplier= depth_multiplier
        self.strides = strides
        self.padding = padding
        self.rates=dilation_rate
        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.kernel_constraint = tf.keras.constraints.get(kernel_constraint)
        self.kernel_regularization = tf.keras.regularizers.get(kernel_regularization)
        # for we are assuming channel last
        self.channel_axis = -1

        # self.output_dim = output_dim

    def build(self, input_shape):
        if input_shape[self.channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')

        input_dim = input_shape[self.channel_axis]
        kernel_shape = self.kernel_size + (input_dim,self.depth_multiplier)
        self.kernel2D = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel2D',constraint =self.kernel_constraint,regularizer=self.kernel_regularization)
        super(DepthwiseDilation2D, self).build(input_shape)

    def call(self, x):
        res=[]
        for di in range(self.depth_multiplier):
            H=tf.nn.dilation2d(x,self.kernel2D[:,:,:,di],strides=(1, ) + self.strides + (1, ),padding=self.padding.upper(),data_format="NHWC",dilations=(1,)+self.rates+(1,))
            res.append(H)
        return tf.concat(res,axis=-1)

    def compute_output_shape(self, input_shape):

        space = input_shape[1:-1]
        new_space = []
        for i in range(len(space)):
            new_dim = conv_utils.conv_output_length(
                space[i],
                self.kernel_size[i],
                padding=self.padding,
                stride=self.strides[i],
                dilation=self.rates[i])
            new_space.append(new_dim)

        return (input_shape[0],) + tuple(new_space) + (self.depth_multiplier,)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'num_filters': self.num_filters,
            'kernel_size': self.kernel_size,
            'depth_multiplier': self.depth_multiplier,
            'strides': self.strides,
            'padding': self.padding,
            'dilation_rate': self.rates,
        })
        return config



xinput = layers.Input(shape=(IMG_SIZE, IMG_SIZE, CHANNELS))
xconv=layers.Conv2D(NFILTERS,(KSIZE,KSIZE),use_bias=False,padding='same')(xinput)
for i in range(NLAYERS):
    xconv = layers.Conv2D(NFILTERS,(3,3),padding='same',activation='relu')(xconv)
xfeatures=layers.GlobalAveragePooling2D()(xconv)
xfeatures=layers.BatchNormalization()(xfeatures)
xfeatures=layers.Dense(SUBSPACE,'relu')(xfeatures)
xfeatures=layers.Dense(SUBSPACE)(xfeatures)
xend=layers.Dense(1,activation='sigmoid')(xfeatures)
modelConv=tf.keras.Model(xinput,xend)
modelConv.summary()
print(modelConv.count_params())

CB1=[tf.keras.callbacks.EarlyStopping(patience=PATIENCE_ES,restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=PATIENCE_RP,min_lr=1e-6),
#    tf.keras.callbacks.CSVLogger(dir_autosave_model_stat+'Conv', separator=',', append=False)
   ]
modelConv.compile(loss="mae", optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=["mse","mae"])
histConv=modelConv.fit(listIm, listY, batch_size=batch_size, epochs=epochs,callbacks=CB1,validation_data=(listImVal, listYVal))


xinput = layers.Input(shape=(IMG_SIZE, IMG_SIZE, CHANNELS))
xconv = DepthwiseDilation2D((KSIZE,KSIZE),depth_multiplier=NFILTERS,padding='same')(xinput)
for i in range(NLAYERS):
    xconv = layers.Conv2D(NFILTERS,(3,3),padding='same',activation='relu')(xconv)
xfeatures=layers.GlobalAveragePooling2D()(xconv)
xfeatures=layers.BatchNormalization()(xfeatures)
xfeatures=layers.Dense(SUBSPACE,activation='relu')(xfeatures)
xfeatures=layers.Dense(SUBSPACE)(xfeatures)
xend=layers.Dense(1,activation='sigmoid')(xfeatures)
modelDil=tf.keras.Model(xinput,xend)
modelDil.summary()
print(modelDil.count_params())

CB2=[tf.keras.callbacks.EarlyStopping(patience=PATIENCE_ES,restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=PATIENCE_RP,min_lr=1e-6),
#    tf.keras.callbacks.CSVLogger(dir_autosave_model_stat+'Dil', separator=',', append=False)
   ]
modelDil.compile(loss="mae", optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=["mse","mae"])
histDil=modelDil.fit(listIm, listY, batch_size=batch_size, epochs=epochs,callbacks=CB2,validation_data=(listImVal, listYVal))



Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 128, 128, 48)      8112      
                                                                 
 conv2d_7 (Conv2D)           (None, 128, 128, 48)      20784     
                                                                 
 global_average_pooling2d_2  (None, 48)                0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_2 (Bat  (None, 48)                192       
 chNormalization)                                                
                                                                 
 dense_6 (Dense)             (None, 12)                588 

29845
Epoch 1/512


2023-10-02 16:25:59.368643: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


127/127 [==============================] - ETA: 0s - loss: 0.1182 - mse: 0.0229 - mae: 0.1182

2023-10-02 16:26:11.136580: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

29845
Epoch 1/512


2023-10-02 16:31:26.216602: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


127/127 [==============================] - ETA: 0s - loss: 0.1290 - mse: 0.0310 - mae: 0.1290

2023-10-02 16:33:56.613492: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


127/127 [==============================] - 161s 1s/step - loss: 0.1290 - mse: 0.0310 - mae: 0.1290 - val_loss: 0.2572 - val_mse: 0.0898 - val_mae: 0.2572 - lr: 0.0010
Epoch 2/512
 24/127 [====>.........................] - ETA: 1:52 - loss: 0.1026 - mse: 0.0181 - mae: 0.1026